# Финальное задание
Рассматривается задача предсказания результата матча в Dota 2 по итогам первых 5 минут игры.

Загрузим необходимые библиотеки и датасет:

In [1]:
import pandas as pd
import numpy as np

In [2]:
features = pd.read_csv('C:\\features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


Удалим столбцы, неизвестные через 5 минут после начала игры (в данном случае это последние 6 столбцов). Также удалим время начала игры, т.к. его не стоит использовать в обучении:

In [3]:
X = features.iloc[:, 1:-6]
X.head()

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,7,11,5,2098,1489,20,0,0,7,67,...,0,35,103,-84,221,3,4,2,2,-52
1,0,42,4,1188,1033,9,0,1,12,49,...,0,-20,149,-84,195,5,4,3,1,-5
2,7,33,4,1319,1270,22,0,0,12,98,...,1,-39,45,-77,221,3,4,3,1,13
3,1,29,4,1779,1056,14,0,0,5,30,...,0,-30,124,-80,184,0,4,2,0,27
4,7,13,4,1431,1090,8,1,0,8,27,...,0,46,182,-80,225,6,3,3,0,-16


## Подготовка данных
Чтобы узнать, в каких колонках пропущены значения, сравним количество элементов в каждой колонке с полным. Если оно не совпадает, значит, какие-то значения пропущены. Выведем имена колонок с пропущенными значениями и процентом пропусков:

In [4]:
length = len(X)
columns_with_NA = []
for i,v in zip(xrange(length), X.count()):
    if v != length:
        columns_with_NA.append((X.columns.values[i], round(100.0-v*100.0/length,1)))
columns_with_NA;

Колонки с пропущенными событиями делятся на два типа.

Те, что связаны с событием "первая кровь" (первое убийство героя в игре):

| Колонка | % пропусков |       
| :--: |:-----------: |
|first_blood_time| 20% |
|first_blood_team| 20% |
|first_blood_player1| 20% |
|first_blood_player2| 45% |

Пропуски в них связаны с тем, что __первого убийства не было в течение первых 5 минут игры__. Из данных видно, что убийство не успело произойти за 5 минут примерно в 20% игр, при этом ещё в 25% игр в убийстве участвовал только один герой (поэтому *first_blood_player2* пустой).

Те, что связаны с покупкой предметов: (с префиксами radiant/dire)

| Колонка | Описание | % пропусков |       
| :--: |:-----------: |:-----------: |
| _bottle_time | покупка Magic Bottle | 16-17% |
| _courier_time | покупка курьера | 0.7% |
| _flying_courier_time | апгрейд курьера | 27-28% |
| _first_ward_time | первый вард | 2% |

Пропуски в них связаны с тем, что __соответсткующие предметы ещё ни разу не покупались__. Видно, что курьера чаще всего покупают в течение первых 5 минут, равно как и варды. А вот сделать апгрейд курьера до летающего успели только в 72% игр. *Magic Bottle* также появляется не в каждой игре.

Заполним пропуски нулями, как предлагается в описании задания. На самом деле, разумнее было бы заменить пропуски с суффиксом *_time* каким-нибудь большим значением (например, час-полтора, что больше средней длительности матча в Dota 2). Сложнее с *first_blood_player 1-2* - эти данные бессмысленны, если "первой крови" ещё не было.

In [5]:
X = X.fillna(0)

Целевая переменная содержится в столбце __radiant_win__. Выделим её в отдельную переменную:

In [6]:
y = features['radiant_win']

## Градиентный бустинг
Попробуем использовать "в лоб" градиентный бустинг.

Импортируем необходимые библиотеки:

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

### Метрика качества
Будем измерять AUC-ROC для кросс-валидации по 5 блокам. Проверим классификаторы с различным числом деревьев:

In [96]:
import time
import datetime

start_time = datetime.datetime.now()

trees_cnts = [10, 20, 30]
scores = {}
for cnt in trees_cnts:
    clf = GradientBoostingClassifier(n_estimators=cnt)
    validator = KFold(length, n_folds=5, shuffle=True)
    scores[cnt] = np.mean(cross_val_score(estimator = clf, X = X, y = y, cv = validator, scoring = 'roc_auc'))

print 'Time elapsed:', datetime.datetime.now() - start_time

In [97]:
scores

{10: 0.66578343249719929, 20: 0.68280458694053037, 30: 0.69011898351149614}

Среднее значение метрики AUC-ROC по блокам кросс-валидации при различном числе деревьев:

|Число деревьев|AUC-ROC|
|:--:|:--:|
|10|66.6%|
|20|68.3%|
|30|69.0%|

Видим, что качество модели не слишком высокое (на уровне 70%), т.е. по общим критерия качество примерно посередине между poor (60-70, D) и fair (70-80, C). При этом 30 деревьев всё ещё недостаточно - зависимость пока ещё монотонная. __Скорее всего, с ростом числа деревьев качество будет продолжать расти.__ При этом обучение занимает длительное время, порядка 10 минут, из них 5 минут проводится кросс-валидация классификатора с 30 деревьями. Поэтому использовать большее число деревьев в градиентном бустинге в данной задаче имеет смысл, но в этом случае желательно ускорить обучение. Можно предложить __сократить число признаков__, скомбинировав некоторые из них в "интегральные" (например, использовать максимальный уровень героя в команде или суммарный уровень всех героев команды, суммарный опыт, суммарное заработанное золото и т.п.).

## Логистическая регрессия
Загрузим необходимые библиотеки:

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV

Так как логистическая регрессия чувствительна к масштабу признаков, нормализуем их:

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Метрика качества и подбор параметров
Нам необходимо подобрать метапараметр регрессии "C", для этого воспользуемся классом *GridSearchCV*. Переберём диапазон C от 10^-5 до 10^6:

In [13]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(length, n_folds=5, shuffle=True)
clf2 = LogisticRegression()
gs = GridSearchCV(clf2, grid, scoring='roc_auc', cv=cv)
gs.fit(X_scaled, y)
print gs.best_estimator_
print gs.best_score_

In [14]:
np.mean(cross_val_score(estimator = LogisticRegression(C=0.01), X = X_scaled, y = y, cv = cv, scoring = 'roc_auc'))

0.71642524066195357

Наилучшим параметром оказался C=0.01, AUC-ROC при этом составила 72%, __чуть лучше__, чем у градиентного бустинга. Сама модель при этом обучается __немного быстрее__, чем градиентный бустинг (по крайней мере, на моём компьютере).

### Модель без категориальных признаков
Попробуем убрать категориальные признаки из выборки:

In [15]:
to_remove = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for col in to_remove:
    X.drop(col,inplace=True,axis=1)

In [16]:
X.head()

,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,r2_gold,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,5,2098,1489,20,0,0,7,3,842,991,...,0,35,103,-84,221,3,4,2,2,-52
1,4,1188,1033,9,0,1,12,4,1596,993,...,0,-20,149,-84,195,5,4,3,1,-5
2,4,1319,1270,22,0,0,12,3,1314,775,...,1,-39,45,-77,221,3,4,3,1,13
3,4,1779,1056,14,0,0,5,2,539,539,...,0,-30,124,-80,184,0,4,2,0,27
4,4,1431,1090,8,1,0,8,2,629,552,...,0,46,182,-80,225,6,3,3,0,-16


In [16]:
scaler = StandardScaler()
X_scaled_filtered = scaler.fit_transform(X)

In [65]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(length, n_folds=5, shuffle=True)
clf2 = LogisticRegression()
gs = GridSearchCV(clf2, grid, scoring='roc_auc', cv=cv)
gs.fit(X_scaled_filtered, y)
print gs.best_estimator_
print gs.best_score_

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.716406150708


In [18]:
np.mean(cross_val_score(estimator = LogisticRegression(C=0.01), X = X_scaled_filtered, y = y, cv = cv, scoring = 'roc_auc'))

0.71641872818942876

После удаления категориальных признаков из модели её качество практически не изменилось, видимо, они не вносили достаточного вклада в классификацию. Видимо, это произошло от того, что мы некорректно трактовали эти признаки как числовые, поэтому их вклад оказался на уровне шума.

### Модель с мешком слов
Вернём признаки в модель и посчитаем, сколько различных героев есть в Dota 2:

In [17]:
X = features.iloc[:, 1:-6]
X = X.fillna(0)
hero_ids = []
picks = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for hero in picks:
    hero_ids.extend(np.unique(X[hero]))
    
print np.unique(hero_ids)
len(np.unique(hero_ids))

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 109 110 112]


108

По-видимому, в игре в режиме Captain's mode доступны __108__ героев, хотя судя по id, всего героев как минимум 112. Попробуем подход с мешком слов для героев:

In [13]:
# N — количество различных героев в выборке
N = 112
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [18]:
X_sacked = np.hstack((X_scaled_filtered,X_pick))

Снова запустим логистическую регрессию, теперь уже на расширенном наборе параметров:

In [19]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(length, n_folds=5, shuffle=True)
clf2 = LogisticRegression()
gs = GridSearchCV(clf2, grid, scoring='roc_auc', cv=cv)
gs.fit(X_sacked, y)
print gs.best_estimator_
print gs.best_score_

LogisticRegression(C=0.10000000000000001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
0.751784307852


In [50]:
np.mean(cross_val_score(estimator = LogisticRegression(C=0.1), X = X_sacked, y = y, cv = cv, scoring = 'roc_auc'))

0.75178430785177963

Видим, что AUC-ROC __вырос на 4%__ - весьма значительный прирост для такого рода модели. Это можно объяснить тем, что мы учли довольно важный аспект игры - win rate для героев. Если кратко, одни герои более полезны в игре, чем другие. Некоторые герои требуют специальной стратегии и поэтому реже приносят победу команде.

Итоговые результаты:

|Модель|AUC-ROC|
|:--:|:--:|
|Raw|71.6%|
|w/o categorical|71.6%|
|Sacked|75.2%|

Следующим шагом было бы хорошо учесть взаимодействие героев с героями команды противника - многие герои имеют "контр-пик", который позволяет с большей вероятностью выиграть против них. Т.е. учёт сочетаний пар героев radiant/dire мог бы ещё помочь увеличить точность модели. Правда, при этом число параметров возрастает до нереальных размеров, и требуется отбор признаков.

## Проверка на тестовой выборке
Загрузим тестовую выборку:

In [20]:
features_test = pd.read_csv('C:\\features_test.csv', index_col='match_id')

Уберём из признаков дату матча и пропуски:

In [23]:
X_test = features_test.iloc[:, 1:]
X_test = X_test.fillna(0)

In [24]:
X_test.head()

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,0,93,4,1103,1089,8,0,1,9,102,...,0,12,247,-86,272,3,4,2,0,118
7,1,20,2,556,570,1,0,0,9,6,...,2,-29,168,-54,0,3,2,2,1,16
10,1,112,2,751,808,1,0,0,13,26,...,1,-22,46,-87,186,1,3,3,0,-34
13,1,27,3,708,903,1,1,1,11,91,...,2,-49,30,-89,210,3,4,2,1,-26
16,1,39,4,1259,661,4,0,0,9,93,...,0,36,180,-86,180,1,3,2,1,-33


Добавим "мешок героев":

In [25]:
# N — количество различных героев в выборке
N = 112
X_pick_test = np.zeros((X_test.shape[0], N))

for i, match_id in enumerate(X_test.index):
    for p in xrange(5):
        X_pick_test[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

Уберём категориальные признаки и нормализуем признаки:

In [26]:
to_remove = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for col in to_remove:
    X_test.drop(col,inplace=True,axis=1)
    
scaler = StandardScaler()
X_scaled_filtered_test = scaler.fit_transform(X_test)

Соберём все признаки в одну матрицу:

In [27]:
X_sacked_test = np.hstack((X_scaled_filtered_test,X_pick_test))

Теперь мы можем получить предсказания нашего классификатора для этих объектов:

In [31]:
y_test = gs.best_estimator_.predict_proba(X_sacked_test)

Интересно, что наша модель подтверждает известный факт, что Radiant в среднем выигрывает чуть чаще, чем Dire (т.к. расположение Рошана и спавнов крипов не совсем симметрично):

In [46]:
y_test = pd.DataFrame(y_test)
print 'Dire wins on average: ', np.round(100.0*np.mean(y_test[0]),2), '% of games'
print 'Radiant wins on average: ', np.round(100.0*np.mean(y_test[1]),2), '% of games'

Dire wins on average:  48.18 % of games
Radiant wins on average:  51.82 % of games


In [48]:
y_test.head()

,0,1
0,0.163961,0.836039
1,0.222292,0.777708
2,0.798198,0.201802
3,0.129079,0.870921
4,0.738465,0.261535


Вероятности выглядят разумно, принадлежат интервалу [0, 1] и сумма вероятностей в строке равна 1, как и должно быть. Минимальная и максимальная вероятности:

In [49]:
print 'Minimal prob.:', min(y_test[1])
print 'Maximal prob.:', max(y_test[1])

Minimal prob.: 0.00850228028092
Maximal prob.: 0.996617118712


## Итоги
В данной задаче логистическая регрессия показала лучший результат и позволила достичь 75% AUC-ROC. Модель можно улучшать далее путём конструирования признаков.